In [44]:
import tensorflow as tf
import numpy as np

In [46]:
# [털, 날개]
x_data = np.array([[0, 0], [1, 0], [1, 1], [0, 0], [0, 0], [0, 1]]) 
# 이후, 종류를 나타내는 레이블(분류값) 데이터를 구성한다.
# 레이블 데이터는 원-핫 인코딩(one-hot encoding) 이라는 특수한 형태로 구성. 각 종류에 해당하는 인덱스 값만 1로 설정
y_data = np.array([
    [1, 0, 0], # 기타
    [0, 1, 0], # 포유류
    [0, 0, 1], # 조류
    [1, 0, 0],
    [1, 0, 0],
    [0, 0, 1]
])

In [47]:
# 신경망 모델 구성. 특징 X와 레이블 Y와의 관계를 알아내는 모델. 실측값을 넣을 것이므로 플레이스홀더로 설정
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

In [49]:
# 가중치와 편향 변수 설정
W = tf.Variable(tf.random_uniform([2, 3], -1., 1.))
b = tf.Variable(tf.zeros([3]))   # 레이블 수인 3개의 요소를 가진 변수로 설정

In [51]:
L = tf.add(tf.matmul(X, W), b)
L = tf.nn.relu(L)
# 62P 그림과 같이 나타낼 수 있다

In [52]:
# 배열 내의 결괏값들을 전체 합이 1이 되도록 만들어 주는 softmax 함수 사용
model = tf.nn.softmax(L)

In [55]:
# 손실함수는 원-핫 인코딩을 이용한 대부분의 모델에서 사용하는 교차 엔트로피 함수 사용
# 예측값과 실제값 사이의 확률분포 차이를 계산한 값
# 손실함수는 비용함수(cost function) 이라고도 함.
cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(model), axis=1))
# 이후 결국 배열 안 값의 평균을 내면 그것이 바로 교차 엔트로피 값이 된다.

In [56]:
# 학습단계. TF가 기본으로 제공하는 경사하강법 최적화 시킴
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train_op = optimizer.minimize(cost)

# 텐서플로 세션 초기화
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

# 앞서 구성한 특징과 레이블 데이터를 이용해 학습을 100번 진행
for step in range(100):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})
    
    # 학습 도중 10번에 한 번씩 손실값 출력
    if (step + 1) % 10 == 0:
        print(step + 1, sess.run(cost, feed_dict={X: x_data, Y: y_data}))

10 0.93513703
20 0.93190783
30 0.9287364
40 0.9256131
50 0.9224534
60 0.91939497
70 0.9163599
80 0.9132971
90 0.9103406
100 0.90740865


In [57]:
# 학습 결과 확인 코드
prediction = tf.argmax(model, axis=1)
target = tf.argmax(Y, axis=1)
print('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print('실제값:', sess.run(target, feed_dict={Y: y_data}))

예측값: [1 2 2 1 1 2]
실제값: [0 1 2 0 0 2]


In [58]:
# 정확도 출력
is_correct = tf.equal(prediction, target)   # 예측값과 실측값을 tf.equal 함수로 비교한 뒤, T/F값으로 나온 결과를 다시
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32)) # tf.cast함수로 0과 1로 바꿔 평균 냄
print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))  

정확도: 33.33


# 정확도를 높이는 심층 신경망(다중 레이어) 구현

In [61]:
# 가중치 설정
W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.))  # 특징, 은닉층의 뉴련수
W2 = tf.Variable(tf.random_uniform([10, 3], -1., 1.))  # 은닉층의 뉴런 수, 분류수

# 편향 설정
b1 = tf.Variable(tf.zeros([10])) # 은닉층의 뉴런 수
b2 = tf.Variable(tf.zeros([3]))  # 분류 수

In [62]:
# 특징 입력값에 첫 번째 가충치와 편향, 활성화 함수 적용
L1 = tf.add(tf.matmul(X, W1), b1)
L1 = tf.nn.relu(L1)

In [63]:
# 출력층 생성. 은닉층에 두 번째 가충지 W2와 편향 b2 적용
model = tf.add(tf.matmul(L1, W2), b2)

In [66]:
# 손실함수 작성. 교차 엔트로피 사용(TF가 기본으로 제공하는)
cost = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=model))

optimizer = tf.train.AdamOptimizer(learning_rate=0.01)  # 최적화 함수로 AdamOptimizer 사용. 사용하는 최적화 함수에 따라 정확도나 학습 속도가 많이 달라짐. 그래디언트 디센트 메소드 보다 보편적으로 성능이 좋다고 알려져 있음.
train_op = optimizer.minimize(cost)

In [67]:
# 앞서 사용한 코드 사용으로 결과 확인

# 텐서플로 세션 초기화
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

# 앞서 구성한 특징과 레이블 데이터를 이용해 학습을 100번 진행
for step in range(100):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})
    
    # 학습 도중 10번에 한 번씩 손실값 출력
    if (step + 1) % 10 == 0:
        print(step + 1, sess.run(cost, feed_dict={X: x_data, Y: y_data}))
        
# 학습 결과 확인 코드
prediction = tf.argmax(model, axis=1)
target = tf.argmax(Y, axis=1)
print('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print('실제값:', sess.run(target, feed_dict={Y: y_data}))

# 정확도 출력
is_correct = tf.equal(prediction, target)   # 예측값과 실측값을 tf.equal 함수로 비교한 뒤, T/F값으로 나온 결과를 다시
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32)) # tf.cast함수로 0과 1로 바꿔 평균 냄
print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))  

10 0.8381664
20 0.6776533
30 0.5088869
40 0.37251988
50 0.27139378
60 0.19284411
70 0.13191079
80 0.08881017
90 0.060732286
100 0.043348443
예측값: [0 1 2 0 0 2]
실제값: [0 1 2 0 0 2]
정확도: 100.00
